# 設置引數
此程式用來製作MVSNet之input

需要不同角度的圖片

In [25]:
import argparse
import sys

parser = argparse.ArgumentParser(description='caculate diff view of object and Make input for MVSNet')

parser.add_argument('--chessboard_width',type=int,default=8,help='bigger size of chessboard')
parser.add_argument('--chessboard_height',type=int,default=5,help='smaller size of chessboard')
parser.add_argument('--grid_size',type=int,default=15,help='edge length of chessboard-grid in real world(unit：mm)')
parser.add_argument('--path',default="./INPUT/scan1001/",help='diff view picture location')
parser.add_argument('--assumed_depth',type=int,default=170,help='in MVS的初始平面(距離相機) (單位：mm)')
parser.add_argument('--interval_depth',type=int,default=2.5,help='每個平面的距離－在算的時候為：初始平面 + depth interval (單位 mm)')
parser.add_argument('--image_type',default="jpg",help='view-img type(jpg)(png)(jepg)')
# parse arguments and check
args = parser.parse_args(args=[])
print(args)

Namespace(chessboard_width=8, chessboard_height=5, grid_size=15, path='./INPUT/scan1001/', assumed_depth=170, interval_depth=2.5, image_type='jpg')


# 前處理，將圖片全部reshape成(1600,1200)並儲存
MVSNet所需之input size

In [26]:
import glob
import os
import cv2

images = glob.glob(os.path.join(args.path,'*.{}'.format(args.image_type)))

for fname in images:
    img = cv2.imread(fname)
    # Resize Image
    img = cv2.resize(img,(1600,1200),interpolation=cv2.INTER_AREA)
    cv2.imwrite("{}".format(fname),img)

# 偵測chessboard並做Camera calibration
ref：https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

In [27]:
import numpy as np
import cv2
import glob
from sympy import pprint
import os

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # 30iter校正、0.001
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((args.chessboard_height*args.chessboard_width,3), np.float32)
objp[:,:2] = np.mgrid[0:args.chessboard_width,0:args.chessboard_height].T.reshape(-1,2)*args.grid_size # "*10"：棋盤方格 10mm x 10mm
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# path = "./Calibration_Img/scan1001/"

images = glob.glob(os.path.join(args.path,'*.{}'.format(args.image_type)))

ret_img = [] #紀錄有被detect到的圖片

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (args.chessboard_width,args.chessboard_height), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        ret_img.append(fname) #有定位到這張圖片
        
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (args.chessboard_width,args.chessboard_height), corners2, ret)
#         cv2.imshow('img', img)
#         cv2.waitKey(0)

# cv2.destroyAllWindows()

print("detected_img_num = ",len(ret_img))

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)  

intrinsic_M = []
intrinsic_M.append(mtx)

Extrinsic_M = []
for img_idx in range(len(ret_img)):
    r = cv2.Rodrigues(rvecs[img_idx])
    Extrinsic_M.append(np.concatenate((r[0],tvecs[img_idx]),axis =1))
print("Extrinsic_M :")
pprint(Extrinsic_M)
print("*"*20)
print("intrinsic_M :")
pprint(intrinsic_M)


detected_img_num =  31
Extrinsic_M :
⎡ [[-4.86912959e-01  8.68306876e-01 -9.46516756e-02 -8.08118666e+01],  [[-5.84
⎢  [-8.32758694e-01 -4.28804684e-01  3.50199231e-01  2.60764056e+01]    [-7.59
⎣ [ 2.63493318e-01  2.49338549e-01  9.31880657e-01  2.90050985e+02]]   [ 2.856

928890e-01  8.09200058e-01 -5.52581242e-02 -7.11845629e+01],  [[-6.48606965e-0
111579e-01 -5.22180119e-01  3.88686934e-01  3.96332550e+01]    [-7.07575069e-0
70796e-01  2.69301299e-01  9.19711371e-01  2.96486940e+02]]   [ 2.80439878e-01

1  7.58821519e-01 -5.91515559e-02 -6.98397696e+01],  [[-7.48579993e-01  6.6301
1 -5.72515985e-01  4.14201603e-01  4.99626529e+01]    [-5.99036340e-01 -6.7258
  3.10508211e-01  9.08261045e-01  2.99933685e+02]]   [ 2.84224309e-01  3.28680

9763e-01 -5.72612019e-03 -5.32058377e+01],  [[-9.17233430e-01  3.98133076e-01 
7305e-01  4.34490253e-01  6.46619922e+01]    [-3.54404996e-01 -8.30672297e-01 
865e-01  9.00658332e-01  3.02912117e+02]]   [ 1.81878899e-01  3.89196080e-01  

 1.31487277

# 將相機參數寫入.txt檔

In [28]:
cam_file = args.path+'cams/'
os.makedirs(cam_file, exist_ok=True)


for img_idx in range(len(ret_img)):
#     os.makedirs(cam_file+'{:0>8}_cam.txt'.format(img_idx), exist_ok=True)
    with open(cam_file+'{:0>8}_cam.txt'.format(img_idx), 'w') as f:
        # write extrinsic
        f.write('extrinsic\n')
        for i in range(3):
            f.write(" ".join(str(Extrinsic_M[img_idx][i])[1:-1].split()))
            f.write("\n")
        f.write("0.0 0.0 0.0 1.0\n")
        f.write("\n")
        
        # write intrinsic
        f.write("intrinsic\n")
        for i in range(3):
            f.write(" ".join(str(intrinsic_M[0][i])[1:-1].split()))
            f.write("\n")
        f.write("\n")
        
        # write the assumed depth and assume interval depth
#         assumed_depth = 170 # 初始平面(距離相機) (單位：mm)
#         interval_depth = 2.5 # 每個平面的距離－在算的時候為：初始平面 + depth interval (單位 mm)
        f.write(str(args.assumed_depth))
        f.write(" ")
        f.write(str(args.interval_depth))
    f.close

# 將input的圖片 distorted 回來並save之

In [29]:
img_file = args.path+'images/'
os.makedirs(img_file, exist_ok=True)

# 將img distorted回來
for idx,fname in enumerate(ret_img):
    img = cv2.imread(fname)
    dst = cv2.undistort(img,mtx,dist,None,mtx)
    
    cv2.imwrite(img_file+'{:0>8}.jpg'.format(idx),dst)

# 製作MVS_Net所需的pair之資料

In [30]:
# pair.txt 存至 path中
with open(args.path+'pair.txt', 'w') as f:
    # 第一行 總圖片數
    f.write(str(len(ret_img)))
    f.write("\n")
    # 依據總圖片數 做 pair 匹配 ， 但正確方法須 稀疏重建，這裡簡化為下一張圖片
    for img_idx in range(len(ret_img)): #對每個圖片進行配對
        f.write("{}\n".format(img_idx)) #圖片ID
        #取 X 張src img
        select_src_num = 10
        f.write("{} ".format(select_src_num)) 
        for i in range(select_src_num):
            f.write("{} ".format((img_idx+i+1)%len(ret_img))) # 配對的cloud point數，但這裡沒真正做稀疏重建，所以為下一張
            f.write("{} ".format(select_src_num-i)) # score # 這裡score為假資料
        f.write("\n")
f.close

<function TextIOWrapper.close()>

In [31]:
# import numpy as np
# a = np.arange(10,14).reshape(2,2)
# np.pad(a,((3,2),(2,5)),'constant',constant_values = (0,0))

In [32]:
# import cv2
# img = cv2.imread("C:/Users/TshongHangY/Desktop/Pytorch_Practice/MVSNet_pytorch-master/Calibration_Img/scan1001/0.jpg")
# print(np.shape(img))

# Resize_img = cv2.resize(img,(1600,1200),interpolation=cv2.INTER_AREA)
# cv2.imshow("Resize",Resize_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# # ===== padding =====
# # 1600 * 1200
# # np.pad(img,((0,),(0,5)),'constant',constant_values = (0,0))
# # np.pad()